 # Desconsiderando os registros que possuam grande concordância ou pouquíssima concordância

### Lançando mão da entropia e monotonicidade sobre **conjunto único**

In [ ]:
import os
import re
import numpy as np
import math
import pandas as pd
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# nao_passaram = 0
# iteracoes = 0

############################################################################################################################
#Definição de funções
############################################################################################################################

#Função para geração do F1 médio
def geraF1(toClass, classificador): #toClass é o conjunto de treinamento
    
    f1 = 0
    std = 0
    coef_var = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    
    kfold = KFold(n_splits=5, random_state=seed)
    
    try:
    
        kfold = StratifiedKFold(n_splits=5, random_state=seed) #Mudar para n-fold #Adicionado depois!
        
    except ValueError:
                
        kfold = KFold(n_splits=5, random_state=seed)
            
    except:
        print('Erro com a validação cruada!')
        print ("Unexpected error:", sys.exc_info()[0])
    
    try:
        cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
        
        f1 = cv_results.mean()
        std = cv_results.std()
        coef_var = std/f1
    except:
        print("ERRO NA VALIDAÇÃO CRUZADA!")
#         print("Conjunto de treinamento:")
#         print(toClass)
        print("Tamanho de toClass: {0}".format(len(toClass)))
        print ("Unexpected error:", sys.exc_info()[0])
        
    
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

#Função para geração do conjunto de treinamento
def geraTrainSet(ct, dir, file1):

    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#Função para geração do conjunto de teste
def geraTestSet(ct, dir, file1):

    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)            
    
############################################################################################################################
#Repetições para os experimentos começam aqui
############################################################################################################################

#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'min' #Estatística para ordenamento
qtd_alg = 23 #Quantidade total de algoritmos
qtd_alg_nd = 2 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
# k = 3 #Tamanho da janela
# janelas = [2,3,4,5]
janelas = [3]
orcamento = 100 #Ou uma porcentagem da base de dados
orcamento_orig = orcamento
tam_min_ct = 20
seed = 500
nAlg = 23

etapa = '2 - AA[dg-arj]'

# dirOrig = "../../csv/conjuntosDS/conjuntosDiverg/"
dirOrig = "../../csv/conjuntosDS/conjuntosDivergAA/"
estat = "../../csv/estatisticaInicialDS.csv"

for k in janelas:
    
    nao_passaram = 0
    iteracoes = 0


    estatisticas = pd.read_csv(estat, index_col=['algoritmosUtilizados', 'etapa', 'permutacao'], sep=';')

    arquivos = [] #Adicionado depois

    for _, _, arquivo in os.walk(dirOrig):
         arquivos.extend(arquivo)   

    for arq in arquivos:

        if '_NEW' in arq:
            print("##################################################################")
            print("Analisando o arquivo: {0}".format(arq))
            print("##################################################################")

            iteracoes += 1

            num = re.sub(r'diverg.*\)', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
            num = num.replace('_NEW.csv','')

            algUtl = re.sub(r'diverg.*\(', r'', arq) #Alterar para fazer a substituição de tudo em uma linha só
            algUtl = re.sub(r'\).*', r'', algUtl) #Alterar para fazer a substituição de tudo em uma linha só
            algUtl = int(algUtl)

            permutacao = int(num)

            linhaAtual = estatisticas.xs((algUtl, '1 - acm diverg', permutacao))    

            ###### Leitura do conjunto de pares conflitantes
            pc = dirOrig+arq

            pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes

            cols = list(pc.columns.values)
            cols.pop(cols.index('duplicata'))
            pc = pc[['duplicata']+cols]

            pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
            pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 
            
            #Criação da coluna de ENTROPIA
            
            #ENTROPIA = −[P+(e) · ln(P+(e)) +P−(e) · ln(P−(e))]

            #Onde, P+(e) é a fração de casos positivos identificados pelo comitê (|CASOS_POSITIVOS|/|COMITÊ|).
            
            pc_aa['entropia'] = -(pc_aa['qtdAlg']/nAlg * np.log(pc_aa['qtdAlg']/nAlg) + ((nAlg - pc_aa['qtdAlg'])/nAlg) * np.log((nAlg - pc_aa['qtdAlg'])/nAlg))

            # pc_aa = pc.loc[:, :'med' ]
            # pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

            duplicata = pc_aa.loc[:, 'duplicata' ]

            #Adicionando a coluna de duplicatas a pc_vetores
            pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

            ###### 

            #Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
            #que aponta o par como possível duplicata 
            #(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
            #e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
#             ndup = pc_aa.loc[pc['qtdAlg'] <= qtd_alg_nd]
#             dup = pc_aa.loc[(pc['qtdAlg'] > qtd_alg_nd) & (pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd - 1))]
            # dup = dup[pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd)]
            
            ###### Ordenamento dos pares pela estatística selecionada
#             ndup = ndup.sort_values(estat_ord) 
#             dup = dup.sort_values(estat_ord, ascending=False)
            
            #O que for considerado prováveis pares óbvios, tanto positivos quanto negativos, 
            #não serão considerados para a geração do conjunto de treino
            pc_aa = pc_aa.loc[(pc['qtdAlg'] > qtd_alg_nd) & (pc['qtdAlg'] <= (qtd_alg - qtd_alg_nd - 1))]
            pc_aa = pc_aa.sort_values(estat_ord) 

            ###### Variáveis para verificar se pode deslizar a janela dentro de dup (ou ndup)

            deslz = len(pc_aa)/k #ALTERAÇÃO AQUI
            deslz
            
#             deslz_dup = len(dup)/k
#             deslz_dup

#             deslz_ndup = len(ndup)/k
#             deslz_ndup

            ###### Validação do tamanho para deslizamento

            #Se não houver espaço suficiente para deslizar as janelas
#             if (deslz_dup < tam_min_ct/2) | (deslz_ndup < tam_min_ct/2):
            if (deslz < tam_min_ct):
                pode_passar = False
                print("Arquivo: {0} não passou!".format(arq))
                nao_passaram += 1
            else:
                pode_passar = True

            ###### Criação do dataframe que armazenará o conjunto de treinamento

            conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

            ###### Povoamento inicial sem aleatoriedade no except

            import sys

            orcamento = 100
            conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

            continua = True
            f1_anterior = 0
            f1_atual = 0
            f1_svm = 0
            f1_dt = 0

            jan_inic_ndup = 0 
            jan_fin_ndup = k

            jan_inic_dup = 0 
            jan_fin_dup = k

            cont = 0

            entrouExcept = False

            continua = True
            
            if(pode_passar):

#             else: #Se não pode passar!

#                 print('ENTROU NO ELSE POR NÃO PASSAR')

                jan_inic_dup = 0 
                jan_fin_dup = k

                jan_inic_ndup = len(pc_aa) - 1 #Fim de pc_aa
                jan_fin_ndup = len(pc_aa) - k


                #se a janela final de ndup for menor que a janela final de dup para

                #Povoamento inicial com 20 pares rotulados (10 de cada)
                while (cont < tam_min_ct) & (continua) & (jan_fin_dup < jan_fin_ndup):

    #                 print("cont < 20: {0} - continua: {1}".format(cont < tam_min_ct, continua))

    #                 print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_dup, jan_fin_ndup))

                    #Desliza-se a janela
                    reexecuta = True

                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

    #                 print("reexecuta: {0} - deslz_ndup >= 1: {1}".format(reexecuta, deslz_ndup >= 1))

                    while (reexecuta) & (deslz >= 1):

    #                     grupo = pc_aa.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe
                        grupo = pc_aa.iloc[jan_fin_ndup:jan_inic_ndup] # three rows of dataframe

                        #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                        #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                        try:

                            id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                            #Só passa pra cá se não lançar except
                            reexecuta = False
                            #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                            vetor_sel = pc_vetores.loc[id_row_sel]
                            conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                            orcamento -= 1


                        except ValueError:

                            print("Atualizando janelas no except")
                            print(grupo)
                            jan_inic_ndup = jan_inic_ndup - k
                            jan_fin_ndup = jan_fin_ndup - k

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
                            print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))


                            reexecuta = True

                        except:
                            print('Eita!')

    #                 print("Possível não-duplicata")
    #                 print(vetor_sel)

                    reexecuta = True

                    #SELEÇÃO DA POSSÍVEL DUPLICATA
                    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    while (reexecuta) & (deslz >= 1):

                        grupo = pc_aa.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                        try:

                            id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                            #Só passa pra cá se não lançar except
                            reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                            #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                            vetor_sel = pc_vetores.loc[id_row_sel]
                            conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                            orcamento -= 1

                        except ValueError:

                            print("Atualizando janelas no except")
                            print(grupo)
                            jan_inic_dup = jan_inic_dup + k
                            jan_fin_dup = jan_fin_dup + k

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
                            print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))


                            reexecuta = True

                        except:
                            print('Eita!')

    #                 print("Possível duplicata")
    #                 print(vetor_sel)

                    #Atualização das janelas
                    jan_inic_ndup = jan_inic_ndup - k
                    jan_fin_ndup = jan_fin_ndup - k

                    jan_inic_dup = jan_inic_dup + k
                    jan_fin_dup = jan_fin_dup + k

                    if(deslz < 1): #Se não houver mais espaço para deslizar as janelas
                        continua = False


                    cont += 1
                
                #FIM DO WHILE DO CONT


                #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
                f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
                f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

                f1_atual = (f1_svm + f1_dt)/2
                std_atual = (std_svm + std_dt)/2
                cv_atual = (cv_svm + cv_dt)/2

                ###### Aplicação das janelas deslizantes após o povoamento incial sem aleatoriedade no except



                # continua = True
                max_local = 0
                igual = 0

    #             print("Início")
    #             print("orcamento: {0}".format(orcamento))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                while continua:

                    f1_anterior = f1_atual

                    reexecuta = True

                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    while (reexecuta) & (deslz >= 1):
                        grupo = pc_aa.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

                        #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
                        #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                        try:

                            id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                            #Só passa pra cá se não lançar except
                            reexecuta = False #Se selecionou um registro no comando anterior, não precisa reexecutar o while
                            #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")
                            vetor_sel = pc_vetores.loc[id_row_sel]
                            conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                            orcamento -= 1

                        except ValueError:

                            print("Atualizando janelas no except")
                            jan_inic_ndup = jan_inic_ndup - k
                            jan_fin_ndup = jan_fin_ndup - k

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            reexecuta = True

                        except:
                            print('Eita!')

    #                 print("Possível não-duplicata")
    #                 print(vetor_sel)

                    reexecuta = True

                    #SELEÇÃO DA POSSÍVEL DUPLICATA
                    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

                    deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                    while (reexecuta) & (deslz >= 1):

                        grupo = pc_aa.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

                        try:

                            id_row_sel = grupo['entropia'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada

                            #Só passa pra cá se não lançar except
                            reexecuta = False
                            #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
                            vetor_sel = pc_vetores.loc[id_row_sel]
                            conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
                            orcamento -= 1

                        except ValueError:

                            print("Atualizando janelas no except")
                            jan_inic_dup = jan_inic_dup + k
                            jan_fin_dup = jan_fin_dup + k

                            deslz -= 1 #Deslizou a janela -> Diminuiu o espaço para ela rodar

                            reexecuta = True

                        except:
                            print('Eita!')

    #                 print("Possível duplicata")
    #                 print(vetor_sel)

                    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
                    f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
                    f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

                    f1_atual = (f1_svm + f1_dt)/2
                    std_atual = (std_svm + std_dt)/2
                    cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação

                    cv_atual = cv_atual*100


                    if orcamento > 0 : #Se ainda tem orçamento pra gastar
                        if cv_atual < 10:
                            if (f1_atual >= f1_anterior) and (igual <= 3): #Selecionam-se novos pares para rotulação

                                if f1_atual == f1_anterior:
                                    igual += 1
                                elif (f1_atual > f1_anterior) and (igual > 0):
                                    igual = 0

                                continua = True
                            elif (f1_atual < f1_anterior):
                                max_local += 1
                                #Se entrar aqui tem que remover os últimos pares que entraram no conjunto de treinamento

                                #IMPORTANTE! Fazer com que esses devem sejam extraídos para o conjunto PM, não apenas descartados
                                conj_treino.drop(conj_treino.tail(2).index,inplace=True)

    #                             print("f1_atual < f1_anterior")
    #                             print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))

                                if (max_local <= 1):
                                    f1_atual = f1_anterior
                                else:
                                    f1_atual = f1_anterior
    #                                 print('O f1 passou a piorar')
    #                                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                                    continua = False
                                    break
                            elif (igual > 3): #Se a medida de qualidade for igual à anterior (por três vezes)
    #                                 print('F1 convergiu!') 
                                    continua = False
                                    break

                    else:
                        continua = False
                        break

                    #Atualização das janelas
                    jan_inic_ndup = jan_inic_ndup - k
                    jan_fin_ndup = jan_fin_ndup - k

                    jan_inic_dup = jan_inic_dup + k
                    jan_fin_dup = jan_fin_dup + k

                    if(deslz < 1): #Se não houver mais espaço para deslizar as janelas
                        continua = False

    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
    #                 print("igual: {0}".format(igual))

                ##### Salvando os arquivos de treino e teste

#                 dirDest = "../../csv/conjuntosDS/treinoTeste/"
                #FIM DO WHILE
    
                abordagem = 'DS'

                iteracao = 1
                inspecoesManuais = orcamento_orig - orcamento

                duplicatas = [i for i in conj_treino.duplicata if i == True]
                duplicatas = duplicatas.count(True)

                nao_duplicatas = [i for i in conj_treino.duplicata if i == False]
                nao_duplicatas = nao_duplicatas.count(False)

                da = linhaAtual['da'].item()
                dm = duplicatas
                ndm = nao_duplicatas

                tp = float(linhaAtual['tp'].item() + dm)
                fp = float(linhaAtual['fp'].item())
                tn = float(linhaAtual['tn'].item())# + ndm) #Retirado
                fn = float(linhaAtual['fn'].item() - dm) #Adicionado

                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                fmeasure = 2*((precision*recall)/(precision+recall))

                #Adicionando valor à última linha
                estatisticas.loc[(algUtl, etapa, permutacao), ['abordagem', 'iteracao', 'inspecoesManuais',
                   'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp',
                   'fp', 'tn', 'fn'] ] = ([abordagem, iteracao, inspecoesManuais,
                   precision, recall, fmeasure, da, dm, ndm, tp, fp, tn, fn])

                dirDest = "../../csv/conjuntosDS/treinoTeste-k"+str(k)+"/"
        #         dirDest = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/conjuntosDS/treinoTeste/"
        #         dirDest = "./arqResult/csv/conjuntosDS/conjuntosDiverg/treinoTeste/"

                #algUtl = str(algUtl).replace('.0','')
                algUtl = str(algUtl)

                geraTrainSet(conj_treino, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

                indicesCT = conj_treino.index.values.tolist()

                geral = pd.concat([pc_vetores,conj_treino]) #Concatenando pc_vetores e conj_treino

                #Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
                conj_teste = geral.drop(indicesCT, axis='rows') 

                geraTestSet(conj_teste, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')




    ############################################################################################################################
    #Estatísticas
    ############################################################################################################################

    # if (pode_passar):

    #Para voltar o dataframe ao normal
    estatisticas = estatisticas.reset_index(level=['algoritmosUtilizados', 'etapa', 'permutacao'])

    estatisticas = estatisticas[['abordagem', 'etapa', 'algoritmosUtilizados', 'permutacao', 'iteracao', 'inspecoesManuais', 'precision', 'recall', 'f-measure', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']]

    estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']] = \
    estatisticas[['algoritmosUtilizados', 'iteracao', 'inspecoesManuais', 'da', 'dm', 'ndm', 'tp', 'fp', 'tn', 'fn']].astype(int)

    dirEst = "../../csv/"

    # Diretório para Windows
    # dirEst = "C:\Users\Diego\Documents\NetBeansProjects\Master-SKYAM\AS\src\csv\\"
    # dirEst = "../../Documents/NetBeansProjects/Master-SKYAM/AS/src/csv/"


    # Diretório para Linux
    # dirEst = "./arqResult/csv/"

    estatisticas.to_csv(dirEst+'estatisticaInicialDS2-DgArj-k'+str(k)+'.csv', sep=';', index=False)   

    if(iteracoes >= 3000):

        print("Não passaram: {0}".format(nao_passaram))